In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
from sklearn import preprocessing
from statistics import mode
from numpy import median
%matplotlib inline

# Import data

In [2]:
%%time
mydata=pd.read_csv('NY property data.csv')

CPU times: user 5.11 s, sys: 712 ms, total: 5.82 s
Wall time: 5.14 s


# Assign modes to zipcodes

In [3]:
def mods(i):
    d = {}
    for x in i:
        d[x] = d.get(x, 0) +1
   
    return max(d, key = lambda k: d[k])

In [4]:
d1=mydata[['ZIP','B','BLOCK','TAXCLASS']]
count=d1.groupby(['B','TAXCLASS','BLOCK']).agg({'ZIP':[mods, 'count']})
count2=d1.groupby(['B','TAXCLASS']).agg({'ZIP':[mods, 'count']})

#count = count.head(20)

In [5]:
mydata.head()

,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,...,0,0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27,...,0,0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709,...,709,564,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793,...,85,551,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323,...,89,57,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR


In [6]:
mydata=mydata.merge(count, on = ['B', 'BLOCK','TAXCLASS'], suffixes = ['_x', '_y'])
mydata = mydata.merge(count2, on = ['B','TAXCLASS'], suffixes = ['', '_z'])

/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [7]:
x = list(mydata.columns)
x

['RECORD',
 'BBLE',
 'B',
 'BLOCK',
 'LOT',
 'EASEMENT',
 'OWNER',
 'BLDGCL',
 'TAXCLASS',
 'LTFRONT',
 'LTDEPTH',
 'EXT',
 'STORIES',
 'FULLVAL',
 'AVLAND',
 'AVTOT',
 'EXLAND',
 'EXTOT',
 'EXCD1',
 'STADDR',
 'ZIP',
 'EXMPTCL',
 'BLDFRONT',
 'BLDDEPTH',
 'AVLAND2',
 'AVTOT2',
 'EXLAND2',
 'EXTOT2',
 'EXCD2',
 'PERIOD',
 'YEAR',
 'VALTYPE',
 "('ZIP', 'mods')",
 "('ZIP', 'count')",
 ('ZIP_z', 'mods'),
 ('ZIP_z', 'count')]

In [8]:
x[-1] = 'widecount'

In [9]:
x[-2] = 'widemode'
x[-3] = 'count'
x[-4] = 'mode'
mydata.columns = x
x

['RECORD',
 'BBLE',
 'B',
 'BLOCK',
 'LOT',
 'EASEMENT',
 'OWNER',
 'BLDGCL',
 'TAXCLASS',
 'LTFRONT',
 'LTDEPTH',
 'EXT',
 'STORIES',
 'FULLVAL',
 'AVLAND',
 'AVTOT',
 'EXLAND',
 'EXTOT',
 'EXCD1',
 'STADDR',
 'ZIP',
 'EXMPTCL',
 'BLDFRONT',
 'BLDDEPTH',
 'AVLAND2',
 'AVTOT2',
 'EXLAND2',
 'EXTOT2',
 'EXCD2',
 'PERIOD',
 'YEAR',
 'VALTYPE',
 'mode',
 'count',
 'widemode',
 'widecount']

In [10]:
%%time
m = mode(mydata['ZIP'])

CPU times: user 16.3 s, sys: 169 ms, total: 16.4 s
Wall time: 16 s


In [11]:
def compare(x):
    
    if x['present']:
     #   print(2)
        if int(x['count'])>= 10:
            return x['mode']
        
        elif not np.isnan(x['widemode']):
            return x['widemode']
        else:
            return m
    return x['ZIP']

In [12]:
#for i in range(len(mydata)):
#    if np.isnan(mydata['ZIP'][i]):
#        if int(mydata['count'][i])>=10:
#            mydata['ZIP'][i]=mydata['mode'][i]
#        else:
#            mydata['ZIP'][i]=mydata['widemode'][i]
#len(mydata[mydata['ZIP'].isna()==False]['ZIP'])

In [13]:
mydata['present'] = mydata['ZIP'].isna()

In [14]:
%%time
mydata['ZIPS']  = mydata.apply(compare, axis = 1)

CPU times: user 20.8 s, sys: 1.25 s, total: 22 s
Wall time: 21.1 s


In [15]:
# check
mydata[np.isnan(mydata['ZIPS'])][['ZIPS']] 

,ZIPS


In [16]:
mydata['ZIP'] = mydata['ZIPS'].apply(lambda x: str(int(x)))
mydata['ZIP3'] = mydata['ZIP'].apply(lambda x: x[:3])
mydata['ZIP4'] = mydata['ZIP'].apply(lambda x: x[:4])

In [17]:
for x in ['BBLE','BLOCK', 'ZIPS','LOT', 'EASEMENT','OWNER','BLDGCL','EXT','EXLAND','EXTOT','EXCD1','STADDR','EXMPTCL','AVLAND2', 'AVTOT2', 'EXLAND2','EXTOT2','EXCD2','PERIOD','YEAR','VALTYPE','mode','count','widemode','widecount','present']:
    del mydata[x]
    
    
    

In [18]:
#mydata_backup=mydata

In [19]:
#mydata=mydata_backup

# FILLING STORIES

In [20]:
def nonzero(x):
    l=[]
    for i in x:
        if i!=0 and np.isnan(i)==False:
            l.append(float(i))
    return np.median(l)

In [21]:
d2=mydata[['TAXCLASS','ZIP','STORIES']]
count=d2.groupby(['ZIP','TAXCLASS']).agg({'STORIES':[nonzero, 'count']})
count2=d2.groupby(['TAXCLASS']).agg({'STORIES':[nonzero, 'count']})
mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'],suffixes = ['_x', '_y'])
mydata = mydata.merge(count2, on = ['TAXCLASS'],suffixes = ['', 'z'])
x = list(mydata.columns)
x[-1] = 'widecount'
x[-2] = 'widemedian'
x[-3] = 'count'
x[-4] = 'median'
mydata.columns = x
m = mydata['STORIES'].median()


/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [22]:
def compare(x):
    
    if x['present']:
     #   print(2)
        if int(x['count'])>= 10:
            return x['median']
        
        elif not np.isnan(x['widemedian']):
            return x['widemedian']
        else:
            return m
    return x['STORIES']
mydata['present'] = mydata['STORIES'].isna()

mydata['STORY']  = mydata.apply(compare, axis = 1)

In [23]:
mydata[np.isnan(mydata['STORY'])][['STORY']] 

,STORY


In [24]:
mydata['STORIES']=mydata['STORY']

In [25]:
for x in ['median', 'count', 'widemedian', 'widecount', 'present', 'STORY']:
    del mydata[x]

In [26]:
mydata.head(5)

,RECORD,B,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH,ZIP3,ZIP4
0,1,1,4,500,1046,50.0,21400000.0,4225500.0,9630000.0,10004,0,0,100,1000
1,2,1,4,27,0,50.0,193800000.0,14310000.0,87210000.0,10004,0,0,100,1000
2,3,1,4,709,564,3.0,104686000.0,39008700.0,47108700.0,10004,709,564,100,1000
3,4,1,4,793,551,2.0,39200000.0,15255000.0,17640000.0,10004,85,551,100,1000
4,5,1,4,323,1260,1.0,272300000.0,121050000.0,122535000.0,10004,89,57,100,1000


In [27]:
#mydata_backup2=mydata

In [28]:
#mydata=mydata_backup2

# Filling FULLVAL

In [29]:
def nonzero(x):
    l=[]
    for i in x:
        if i!=0 and np.isnan(i)==False:
            l.append(i)
    return np.median(l)

In [30]:
%%time
d3=mydata[['TAXCLASS','ZIP','ZIP3','FULLVAL']]
count=d3.groupby(['ZIP','TAXCLASS']).agg({'FULLVAL':[nonzero, 'count']})
count2=d3.groupby(['ZIP3','TAXCLASS']).agg({'FULLVAL':[nonzero]})
# After groupping by zip3 and taxclass, there are some groups have less than 5 records and the median is NaN
# Thus, group by taxclass only to avoid NaN
count3=d3.groupby(['TAXCLASS']).agg({'FULLVAL':[nonzero]})
mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'],suffixes = ['_x', '_y'])
mydata = mydata.merge(count2, on = ['ZIP3','TAXCLASS'],suffixes = ['', 'z'])
mydata = mydata.merge(count3, on = ['TAXCLASS'])
x = list(mydata.columns)
x[-1] = 'widestmedian'
x[-2] = 'widermedian'
x[-3] = 'count'
x[-4] = 'median'
mydata.columns = x
m=mydata['FULLVAL'].median()

/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


CPU times: user 10.6 s, sys: 354 ms, total: 11 s
Wall time: 8.02 s


/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [31]:

def compare(x):
    
    if (x['present']==True) or (float(x['FULLVAL'])==0):
     #   print(2)
        if int(x['count'])>= 10 and (not np.isnan(x['median'])):
            return x['median']
        elif not np.isnan(x['widermedian']):
            return x['widermedian']
        elif not np.isnan(x['widestmedian']):
            return x['widestmedian']
        else:
            return m
    return x['FULLVAL']
mydata['present'] = mydata['FULLVAL'].isna()

In [32]:
%%time
mydata['FULLVAL2']  = mydata.apply(compare, axis = 1)

CPU times: user 25 s, sys: 541 ms, total: 25.5 s
Wall time: 24.8 s


In [33]:
mydata[np.isnan(mydata['FULLVAL2'])][['FULLVAL2']] 

,FULLVAL2


In [34]:
mydata['FULLVAL']=mydata['FULLVAL2']
for x in ['median', 'count', 'widermedian', 'widestmedian', 'present','FULLVAL2']:
    del mydata[x]

In [35]:
mydata.head(5)

,RECORD,B,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH,ZIP3,ZIP4
0,1,1,4,500,1046,50.0,21400000.0,4225500.0,9630000.0,10004,0,0,100,1000
1,2,1,4,27,0,50.0,193800000.0,14310000.0,87210000.0,10004,0,0,100,1000
2,3,1,4,709,564,3.0,104686000.0,39008700.0,47108700.0,10004,709,564,100,1000
3,4,1,4,793,551,2.0,39200000.0,15255000.0,17640000.0,10004,85,551,100,1000
4,5,1,4,323,1260,1.0,272300000.0,121050000.0,122535000.0,10004,89,57,100,1000


In [36]:
#mydata_backup3=mydata

In [37]:
#mydata=mydata_backup3

# Filling AVLAND

In [38]:
d4=mydata[['TAXCLASS','ZIP','ZIP3','AVLAND']]
count=d4.groupby(['ZIP','TAXCLASS']).agg({'AVLAND':[nonzero, 'count']})
count2=d4.groupby(['ZIP3','TAXCLASS']).agg({'AVLAND':[nonzero]})
# After groupping by zip3 and taxclass, there are some groups have less than 5 records and the median is NaN
# Thus, group by taxclass only to avoid NaN
count3=d4.groupby(['TAXCLASS']).agg({'AVLAND':[nonzero]})
mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'],suffixes = ['_x', '_y'])
mydata = mydata.merge(count2, on = ['ZIP3','TAXCLASS'],suffixes = ['', 'z'])
mydata = mydata.merge(count3, on = ['TAXCLASS'])
x = list(mydata.columns)
x[-1] = 'widestmedian'
x[-2] = 'widermedian'
x[-3] = 'count'
x[-4] = 'median'
mydata.columns = x
m=mydata['AVLAND'].median()

/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(

In [39]:

def compare(x):
    
    if (x['present']==True) or (float(x['AVLAND'])==0):
     #   print(2)
        if int(x['count'])>= 10 and (not np.isnan(x['median'])):
            return x['median']
        elif not np.isnan(x['widermedian']):
            return x['widermedian']
        elif not np.isnan(x['widestmedian']):
            return x['widestmedian']
        else:
            return m
    return x['AVLAND']
mydata['present'] = mydata['AVLAND'].isna()

mydata['AVLAND2']  = mydata.apply(compare, axis = 1)

In [40]:
mydata[np.isnan(mydata['AVLAND2'])][['AVLAND2']] 

,AVLAND2


In [41]:
mydata['AVLAND']=mydata['AVLAND2']

In [42]:
for x in [ 'median', 'count', 'widermedian', 'widestmedian', 'present', 'AVLAND2']:
    del mydata[x]

In [43]:
mydata.head(5)

,RECORD,B,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH,ZIP3,ZIP4
0,1,1,4,500,1046,50.0,21400000.0,4225500.0,9630000.0,10004,0,0,100,1000
1,2,1,4,27,0,50.0,193800000.0,14310000.0,87210000.0,10004,0,0,100,1000
2,3,1,4,709,564,3.0,104686000.0,39008700.0,47108700.0,10004,709,564,100,1000
3,4,1,4,793,551,2.0,39200000.0,15255000.0,17640000.0,10004,85,551,100,1000
4,5,1,4,323,1260,1.0,272300000.0,121050000.0,122535000.0,10004,89,57,100,1000


In [44]:
mydata_backup4=mydata

# Filling AVTOT

In [45]:
d4=mydata[['TAXCLASS','ZIP','ZIP3','AVTOT']]
count=d4.groupby(['ZIP','TAXCLASS']).agg({'AVTOT':[nonzero, 'count']})
count2=d4.groupby(['ZIP3','TAXCLASS']).agg({'AVTOT':[nonzero]})
# After groupping by zip3 and taxclass, there are some groups have less than 5 records and the median is NaN
# Thus, group by taxclass only to avoid NaN
count3=d4.groupby(['TAXCLASS']).agg({'AVTOT':[nonzero]})
mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'],suffixes = ['_x', '_y'])
mydata = mydata.merge(count2, on = ['ZIP3','TAXCLASS'],suffixes = ['', 'z'])
mydata = mydata.merge(count3, on = ['TAXCLASS'])
x = list(mydata.columns)
x[-1] = 'widestmedian'
x[-2] = 'widermedian'
x[-3] = 'count'
x[-4] = 'median'
mydata.columns = x
m=mydata['AVTOT'].median()

/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(

In [46]:

def compare(x):
    
    if (x['present']==True) or (float(x['AVTOT'])==0):
     #   print(2)
        if int(x['count'])>= 10 and (not np.isnan(x['median'])):
            return x['median']
        elif not np.isnan(x['widermedian']):
            return x['widermedian']
        elif not np.isnan(x['widestmedian']):
            return x['widestmedian']
        else:
            return m
    return x['AVTOT']

mydata['present'] = mydata['AVTOT'].isna()
mydata['AVTOT2']  = mydata.apply(compare, axis = 1)

In [47]:
mydata[np.isnan(mydata['AVTOT2'])][['AVTOT2']] 

,AVTOT2


In [48]:
mydata['AVTOT']=mydata['AVTOT2']
for x in [ 'median', 'count', 'widermedian', 'widestmedian', 'present', 'AVTOT2']:
    del mydata[x]

In [49]:
mydata.head(5)

,RECORD,B,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH,ZIP3,ZIP4
0,1,1,4,500,1046,50.0,21400000.0,4225500.0,9630000.0,10004,0,0,100,1000
1,2,1,4,27,0,50.0,193800000.0,14310000.0,87210000.0,10004,0,0,100,1000
2,3,1,4,709,564,3.0,104686000.0,39008700.0,47108700.0,10004,709,564,100,1000
3,4,1,4,793,551,2.0,39200000.0,15255000.0,17640000.0,10004,85,551,100,1000
4,5,1,4,323,1260,1.0,272300000.0,121050000.0,122535000.0,10004,89,57,100,1000


In [50]:
#mydata_backup5=mydata

# Filling LTFRONT

In [51]:
d5=mydata[['TAXCLASS','ZIP','B','LTFRONT']]
count=d5.groupby(['ZIP','TAXCLASS']).agg({'LTFRONT':[nonzero, 'count']})
count2=d5.groupby(['B','TAXCLASS']).agg({'LTFRONT':[nonzero]})
mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'],suffixes = ['_x', '_y'])
mydata = mydata.merge(count2, on = ['B','TAXCLASS'],suffixes = ['', 'z'])

x = list(mydata.columns)
x[-1] = 'widemedian'
x[-2] = 'count'
x[-3] = 'median'
mydata.columns = x
m=mydata['LTFRONT'].median()

/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [52]:
def compare(x):
    
    if (x['present']==True) or (float(x['LTFRONT'])==0):
     #   print(2)
        if int(x['count'])>= 20 and (not np.isnan(x['median'])):
            return x['median']
        elif not np.isnan(x['widemedian']):
            return x['widemedian']
        else:
            return m
    return x['LTFRONT']

mydata['present'] = mydata['LTFRONT'].isna()
mydata['LTFRONT2']  = mydata.apply(compare, axis = 1)

In [53]:
#check
mydata[np.isnan(mydata['LTFRONT2'])][['LTFRONT2']] 

,LTFRONT2


In [54]:
mydata['LTFRONT']=mydata['LTFRONT2']
for x in [ 'median', 'count', 'widemedian', 'present', 'LTFRONT2']:
    del mydata[x]

In [55]:
mydata.head(5)

,RECORD,B,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH,ZIP3,ZIP4
0,1,1,4,500.0,1046,50.0,21400000.0,4225500.0,9630000.0,10004,0,0,100,1000
1,2,1,4,27.0,0,50.0,193800000.0,14310000.0,87210000.0,10004,0,0,100,1000
2,3,1,4,709.0,564,3.0,104686000.0,39008700.0,47108700.0,10004,709,564,100,1000
3,4,1,4,793.0,551,2.0,39200000.0,15255000.0,17640000.0,10004,85,551,100,1000
4,5,1,4,323.0,1260,1.0,272300000.0,121050000.0,122535000.0,10004,89,57,100,1000


In [56]:
#mydata_backup6=mydata

In [57]:
#mydata=mydata_backup6

# Filling LTDEPTH

In [58]:
d6=mydata[['TAXCLASS','ZIP','B','LTDEPTH']]
count=d6.groupby(['ZIP','TAXCLASS']).agg({'LTDEPTH':[nonzero, 'count']})
count2=d6.groupby(['B','TAXCLASS']).agg({'LTDEPTH':[nonzero]})
mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'],suffixes = ['_x', '_y'])
mydata = mydata.merge(count2, on = ['B','TAXCLASS'],suffixes = ['', 'z'])

x = list(mydata.columns)
x[-1] = 'widemedian'
x[-2] = 'count'
x[-3] = 'median'
mydata.columns = x
m=mydata['LTDEPTH'].median()

/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [59]:
def compare(x):
    
    if (x['present']==True) or (float(x['LTDEPTH'])==0):
     #   print(2)
        if int(x['count'])>= 20 and (not np.isnan(x['median'])):
            return x['median']
        elif not np.isnan(x['widemedian']):
            return x['widemedian']
        else:
            return m
    return x['LTDEPTH']

mydata['present'] = mydata['LTDEPTH'].isna()
mydata['LTDEPTH2']  = mydata.apply(compare, axis = 1)

In [60]:
#check
mydata[np.isnan(mydata['LTDEPTH2'])][['LTDEPTH2']] 

,LTDEPTH2


In [61]:
mydata['LTDEPTH']=mydata['LTDEPTH2']
for x in [ 'median', 'count', 'widemedian', 'present', 'LTDEPTH2']:
    del mydata[x]
mydata.head(5)

,RECORD,B,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH,ZIP3,ZIP4
0,1,1,4,500.0,1046.0,50.0,21400000.0,4225500.0,9630000.0,10004,0,0,100,1000
1,2,1,4,27.0,207.0,50.0,193800000.0,14310000.0,87210000.0,10004,0,0,100,1000
2,3,1,4,709.0,564.0,3.0,104686000.0,39008700.0,47108700.0,10004,709,564,100,1000
3,4,1,4,793.0,551.0,2.0,39200000.0,15255000.0,17640000.0,10004,85,551,100,1000
4,5,1,4,323.0,1260.0,1.0,272300000.0,121050000.0,122535000.0,10004,89,57,100,1000


In [62]:
#mydata_backup7=mydata

# Filling BLDFRONT

In [63]:
d7=mydata[['TAXCLASS','ZIP','B','BLDFRONT']]
count=d7.groupby(['ZIP','TAXCLASS']).agg({'BLDFRONT':[nonzero, 'count']})
count2=d7.groupby(['B','TAXCLASS']).agg({'BLDFRONT':[nonzero]})
mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'],suffixes = ['_x', '_y'])
mydata = mydata.merge(count2, on = ['B','TAXCLASS'],suffixes = ['', 'z'])

x = list(mydata.columns)
x[-1] = 'widemedian'
x[-2] = 'count'
x[-3] = 'median'
mydata.columns = x
m=mydata['BLDFRONT'].median()

/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [64]:
def compare(x):
    
    if (x['present']==True) or (float(x['BLDFRONT'])==0):
     #   print(2)
        if int(x['count'])>= 20 and (not np.isnan(x['median'])):
            return x['median']
        elif not np.isnan(x['widemedian']):
            return x['widemedian']
        else:
            return m
    return x['BLDFRONT']

mydata['present'] = mydata['BLDFRONT'].isna()
mydata['BLDFRONT2']  = mydata.apply(compare, axis = 1)

In [65]:
mydata[np.isnan(mydata['BLDFRONT2'])][['BLDFRONT2']] 

,BLDFRONT2


In [66]:
mydata['BLDFRONT']=mydata['BLDFRONT2']
for x in [ 'median', 'count', 'widemedian', 'present', 'BLDFRONT2']:
    del mydata[x]
mydata.head(5)

,RECORD,B,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH,ZIP3,ZIP4
0,1,1,4,500.0,1046.0,50.0,21400000.0,4225500.0,9630000.0,10004,152.0,0,100,1000
1,2,1,4,27.0,207.0,50.0,193800000.0,14310000.0,87210000.0,10004,152.0,0,100,1000
2,3,1,4,709.0,564.0,3.0,104686000.0,39008700.0,47108700.0,10004,709.0,564,100,1000
3,4,1,4,793.0,551.0,2.0,39200000.0,15255000.0,17640000.0,10004,85.0,551,100,1000
4,5,1,4,323.0,1260.0,1.0,272300000.0,121050000.0,122535000.0,10004,89.0,57,100,1000


In [67]:
#mydata_backup8=mydata

In [68]:
#mydata=mydata_backup8

# Filling BLDDEPTH

In [69]:
%%time
d8=mydata[['TAXCLASS','ZIP','B','BLDDEPTH']]
count=d8.groupby(['ZIP','TAXCLASS']).agg({'BLDDEPTH':[nonzero, 'count']})
count2=d8.groupby(['B','TAXCLASS']).agg({'BLDDEPTH':[nonzero]})
mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'],suffixes = ['_x', '_y'])
mydata = mydata.merge(count2, on = ['B','TAXCLASS'],suffixes = ['', 'z'])

x = list(mydata.columns)
x[-1] = 'widemedian'
x[-2] = 'count'
x[-3] = 'median'
mydata.columns = x
m=mydata['BLDDEPTH'].median()
def compare(x):
    
    if (x['present']==True) or (float(x['BLDDEPTH'])==0):
     #   print(2)
        if int(x['count'])>= 20 and (not np.isnan(x['median'])):
            return x['median']
        elif not np.isnan(x['widemedian']):
            return x['widemedian']
        else:
            return m
    return x['BLDDEPTH']

mydata['present'] = mydata['BLDDEPTH'].isna()
mydata['BLDDEPTH2']  = mydata.apply(compare, axis = 1)

/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/liana/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/liana/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


CPU times: user 33.6 s, sys: 766 ms, total: 34.4 s
Wall time: 31 s


In [70]:
mydata[np.isnan(mydata['BLDDEPTH2'])][['BLDDEPTH2']] 

,BLDDEPTH2


In [71]:
mydata['BLDDEPTH']=mydata['BLDDEPTH2']
for x in [ 'median', 'count', 'widemedian', 'present', 'BLDDEPTH2']:
    del mydata[x]

In [72]:
mydata.head(5)

,RECORD,B,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH,ZIP3,ZIP4
0,1,1,4,500.0,1046.0,50.0,21400000.0,4225500.0,9630000.0,10004,152.0,207.0,100,1000
1,2,1,4,27.0,207.0,50.0,193800000.0,14310000.0,87210000.0,10004,152.0,207.0,100,1000
2,3,1,4,709.0,564.0,3.0,104686000.0,39008700.0,47108700.0,10004,709.0,564.0,100,1000
3,4,1,4,793.0,551.0,2.0,39200000.0,15255000.0,17640000.0,10004,85.0,551.0,100,1000
4,5,1,4,323.0,1260.0,1.0,272300000.0,121050000.0,122535000.0,10004,89.0,57.0,100,1000


In [73]:
mydata.to_csv('Filled_data.csv')

In [531]:
# We are done!